In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from intvalpy import lineqs
import sympy
import numpy as np
import os
import json

# Mixed Integer Linear Program (MILP)
$$
\begin{align}
&\min_{x} && c^T x\\
&\text{s.t.} && Ax \leq b\\
& && x \geq 0\\
& && x_i \in \mathbb{Z}, i \in I
\end{align}
$$

In [2]:
def run_cutting_planes():
    # Call Cutting Planes Binary
    cmd = './bin/bar --option --otheroption.jpg'
    b = os.popen(cmd).read()
    print(b)

def load_data():
    return json.load(open('test.json'))

data = load_data()
print(data['system'])

# Access the original system data
d = data['system']
n, m = d['n'], d['m']
A = np.array(d['A'], dtype=np.float64)
b = np.array(d['b'], dtype=np.float64)
c = np.array(d['c'], dtype=np.float64)
I = np.array(d['I'], dtype=np.uint32)
assert(m == A.shape[0] and n == A.shape[1] and m == b.shape[0] and n == c.shape[0])
assert(n == 2)

{'n': 2, 'm': 4, 'c': [0, -1], 'A': [[3, 2], [-3, 2], [-1, 0], [0, -1]], 'b': [6, 0, 0, 0], 'I': [0, 1]}


In [15]:
vertices = lineqs(-A, -b, show=False)
print(vertices)


[[1.  1.5]
 [2.  0. ]
 [0.  0. ]]
